In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Getting Started with Merlin Models: Develop a Model for MovieLens

## Overview

[Merlin Models](https://github.com/NVIDIA-Merlin/models/) is a library for training recommender models. Merlin Models let Data Scientists and ML Engineers easily train standard RecSys models on their own dataset, getting GPU-accelerated models with best practices baked into the library. This will also let researchers to build custom models by incorporating standard components of deep learning recommender models, and then benchmark their new models on example offline datasets. Merlin Models is part of the [Merlin open source framework](https://developer.nvidia.com/nvidia-merlin).

Core features are:
- Many different recommender system architectures (tabular, two-tower, sequential) or tasks (binary, multi-class classification, multi-task)
- Flexible APIs targeted to both production and research
- Deep integration with NVIDIA Merlin platform, including NVTabular for ETL and Merlin Systems model serving


### Learning objectives

- Training [Facebook's DLRM model](https://arxiv.org/pdf/1906.00091.pdf) very easily with our high-level API.
- Understanding Merlin Models high-level API

## Downloading and preparing the dataset

In [2]:
import merlin.models.tf as mm

from merlin.datasets.entertainment import get_movielens

2022-04-11 13:48:01.542462: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-11 13:48:05.894862: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16255 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:0b:00.0, compute capability: 7.0
2022-04-11 13:48:05.896120: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 29912 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:85:00.0, compute capability: 7.0
2022-04-11 13:48:05.897123: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0

We provide the `get_movielens()` function as a convenience to download the dataset, perform simple preprocessing, and split the data into training and validation datasets.

In [3]:
train, valid = get_movielens(variant="ml-1m")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-04-11 13:48:06.783362: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 16255 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:0b:00.0, compute capability: 7.0
2022-04-11 13:48:06.784162: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:1 with 29912 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:85:00.0, compute capability: 7.0
2022-04-11 13:48:06.784949: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:2 with 29912 MB memory:  -> device: 2, name: Tesla V100-SXM2-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0
2022-04-11 13:48:06.785781: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:3 with 29912 MB memory:  -> device: 3, name: Tesla V100-SXM2-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0
downloading ml-1m.zip: 5.93MB [00:02, 2.75MB/s]                                 
unz

FileNotFoundError: [Errno 2] No such file or directory: '/root/merlin-models-data/movielens/users.dat'

## Training the DLRM Model with Merlin Models

We define the DLRM model, whose prediction task is a binary classification. From the `schema`, the categorical features are identified (and embedded) and the target column is also automatically inferred, because of the schema tags. We talk more about the schema in the next [example notebook (02)](02-Merlin-Models-and-NVTabular-integration.ipynb),

In [4]:
model = mm.DLRMModel(
    train.schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(train.schema),
)

model.compile(optimizer="adam")

Next, we train the model.

In [5]:
model.fit(train, batch_size=1024)

2022-04-05 14:47:29.832785: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


782/782 [==============================] - 18s 18ms/step - rating_binary/binary_classification_task/precision: 0.7286 - rating_binary/binary_classification_task/recall: 0.8216 - rating_binary/binary_classification_task/binary_accuracy: 0.7212 - rating_binary/binary_classification_task/auc: 0.7841 - loss: 0.5474 - regularization_loss: 0.0000e+00 - total_loss: 0.5474


We evaluate the model...

In [6]:
metrics = model.evaluate(valid, batch_size=1024, return_dict=True)

2022-04-05 14:47:49.426085: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: cond/then/_0/cond/cond/branch_executed/_170


196/196 [==============================] - 4s 12ms/step - rating_binary/binary_classification_task/precision: 0.7350 - rating_binary/binary_classification_task/recall: 0.8234 - rating_binary/binary_classification_task/binary_accuracy: 0.7285 - rating_binary/binary_classification_task/auc: 0.7936 - loss: 0.5388 - regularization_loss: 0.0000e+00 - total_loss: 0.5388


... and check the evaluation metrics. We use by default typical binary classification metrics -- Precision, Recall, Accuracy and AUC. But you can also provide your own metrics list by setting `BinaryClassificationTask(..., metrics=[])`.

In [7]:
metrics

{'rating_binary/binary_classification_task/precision': 0.7349662184715271,
 'rating_binary/binary_classification_task/recall': 0.8234366178512573,
 'rating_binary/binary_classification_task/binary_accuracy': 0.7284606695175171,
 'rating_binary/binary_classification_task/auc': 0.7936176657676697,
 'loss': 0.5581252574920654,
 'regularization_loss': 0.0,
 'total_loss': 0.5581252574920654}

## Conclusion

Merlin Models enables users to define and train a deep learning recommeder model with only 3 commands.

```python
model = mm.DLRMModel(
    train.schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(
        train.schema.select_by_tag(Tags.TARGET).column_names[0]
    ),
)
model.compile(optimizer="adam")
model.fit(train, batch_size=1024)
```

## Next steps

In the next example notebooks, we will show how the integration with NVTabular and how to explore different recommender models.